# Notebook 3 - Machine Learning Fraud Detection
**Objectif :** Entraîner le GBTClassifier, mieux adapté aux données déséquilibrées, et utiliser l'AUC-PR pour une évaluation pertinente.

In [14]:
# 🔹 Étape 1 : Initialisation Spark et Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create Spark Session
spark = SparkSession.builder \
    .appName("Fraud-ML") \
    .getOrCreate()

print(f"✅ Spark {spark.version} session created")

✅ Spark 3.5.0 session created


In [15]:
# 🔹 Étape 2 : Chargement des données Parquet

# Chemin de base HDFS (hôte:port)
hdfs_base_path = "hdfs://namenode:8020/data"

# Chargement des données Parquet
train_df = spark.read.parquet(f"{hdfs_base_path}/train")
test_df = spark.read.parquet(f"{hdfs_base_path}/test")

# Vérification du schéma (pour s'assurer que 'features' et 'label' sont présents)
print("Schéma du DataFrame d'entraînement :")
train_df.printSchema()

# Affichage des décomptes
print(f"\nNombre d'enregistrements dans Train: {train_df.count()}")
print(f"Nombre d'enregistrements dans Test: {test_df.count()}")

Schéma du DataFrame d'entraînement :
root
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)


Nombre d'enregistrements dans Train: 228045
Nombre d'enregistrements dans Test: 56762


In [16]:
# 🔹 Étape 3 : Entraînement du Modèle GBT

print("--- Entraînement du GBTClassifier (les données sont déjà préparées) ---")

# 1. Définition du modèle
gbt = GBTClassifier(
    featuresCol='features', 
    labelCol='label', 
    maxIter=10 # Vous pouvez ajuster cette valeur si l'entraînement est trop long ou si la précision n'est pas suffisante
)

# 2. Entraînement
gbt_model = gbt.fit(train_df)

print("Modèle GBT entraîné avec succès.")

--- Entraînement du GBTClassifier (les données sont déjà préparées) ---
Modèle GBT entraîné avec succès.


In [17]:
# 🔹 Étape 4 : Évaluation du Modèle GBT 

print("--- Prédictions et Évaluation ---")

# Imports de l'API moderne
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import lit

# 1. Faire des prédictions sur le jeu de test
predictions = gbt_model.transform(test_df)

# --- 2. Calculer l'AUC (Area Under ROC) ---
evaluator_auc = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)
auc = evaluator_auc.evaluate(predictions)
print(f"\nArea Under ROC (AUC) sur le jeu de test : {auc:.4f}")
print("-" * 40)

# --- 3. Calculer les métriques de classification (Précision, Rappel, F1) ---
evaluator_multi = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction"
)

# Calculer la Précision pour la classe de Fraude (label=1)
precision_fraud = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "weightedPrecision"})
recall_fraud = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "weightedRecall"})
f1_score_weighted = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "weightedFMeasure"})

print(f"Précision Pondérée (Weighted Precision) : {precision_fraud:.4f}")
print(f"Rappel Pondéré (Weighted Recall) : {recall_fraud:.4f}")
print(f"F1-Score Pondéré (Weighted F1-Score) : {f1_score_weighted:.4f}")

# --- 4. Afficher la matrice de confusion (pour voir le détail) ---
print("\nMatrice de Confusion (Seuil 0.5):")
predictions.groupBy("label", "prediction").count().show()

--- Prédictions et Évaluation ---

Area Under ROC (AUC) sur le jeu de test : 0.9709
----------------------------------------
Précision Pondérée (Weighted Precision) : 0.9993
Rappel Pondéré (Weighted Recall) : 0.9993
F1-Score Pondéré (Weighted F1-Score) : 0.9993

Matrice de Confusion (Seuil 0.5):
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|56653|
|    1|       1.0|   69|
|    0|       1.0|   17|
|    1|       0.0|   23|
+-----+----------+-----+



In [18]:
# 🔹 Étape 5 : Prédictions sur le test set
predictions = gbt_model.transform(test_df)
print("--- Aperçu des prédictions ---")
predictions.select("features", "label", "prediction", "probability").show(5, truncate=False)

--- Aperçu des prédictions ---
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                          

In [19]:
# 🔹 Étape 5 & 6 (Final) : Évaluation et Sauvegarde

# Import pour la sauvegarde (correction du NameError)
import time 

# 1. Prédictions
predictions = gbt_model.transform(test_df)
print("--- Aperçu des prédictions ---")
predictions.select("features", "label", "prediction", "probability").show(5, truncate=False)

# 2. Évaluation des Métriques
print("\n--- Évaluation des Métriques ---")

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")

# AUC-ROC
roc_auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
print(f"ROC AUC: {roc_auc:.4f}")

# AUC-PR (Plus pertinent pour l'imbalance)
pr_auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
print(f"PR AUC (Area Under PR): {pr_auc:.4f}")

# 3. Sauvegarde du Modèle GBT dans HDFS
import time

timestamp = int(time.time())
# CORRECTION : Utiliser un sous-chemin du répertoire de données où l'écriture est autorisée.
HDFS_SAVE_PATH = f"hdfs://namenode:8020/data/models/fraude_gbt_final_{timestamp}" 

print(f"\n--- Sauvegarde du Modèle GBT dans HDFS : {HDFS_SAVE_PATH} ---")

try:
    # Sauvegarde le modèle GBT
    gbt_model.write().overwrite().save(HDFS_SAVE_PATH)
    print("Sauvegarde réussie dans HDFS.")
    
except Exception as e:
    print(f"ÉCHEC CRITIQUE de sauvegarde HDFS : {e}")
    # Si cette nouvelle tentative échoue, il y a un problème de configuration HDFS plus large.
    raise e

# Définir le chemin final pour l'étape de streaming ou GraphX
FINAL_MODEL_PATH = HDFS_SAVE_PATH
print(f"\n Le chemin FINAL à utiliser pour la suite est : {FINAL_MODEL_PATH}")

--- Aperçu des prédictions ---
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                          